In [1]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [2]:
# dataloader

def load_data(filename):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [3]:
mincount = 1
def remove_rare_words(data, vocab, mincount):
    ## FILL CODE
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []
    for lst in data:
        sentence = []
        for word in lst:
            if (word not in vocab) or (vocab[word] < mincount):
                word = '<unk>'
            sentence.append(word)
        data_with_unk.append(sentence)
  
    
    return data_with_unk

In [4]:
# LOAD DATA

print("load training set")
train_data, vocab = load_data("train.txt")

## FILL CODE
# Same as bigram.py
train_data = remove_rare_words(train_data, vocab, mincount)

print("load validation set")
valid_data, _ = load_data("valid.txt")
## FILL CODE
# Same as bigram.py
valid_data = remove_rare_words(valid_data, vocab, mincount)

load training set
load validation set


In [5]:
def build_ngram(data, n):
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        
        total_number_words += len(sentence)
        for ele1 in range(1, n+1):
            for ele2 in range(len(sentence)- ele1 + 1):
                counts[sentence[ele2:(ele1 + ele2-1)]][sentence[ele1 + ele2 - 1]] += 1

    prob  = defaultdict(lambda: defaultdict(lambda: 0.0))
    ## FILL CODE
    # Build the probabilities from the counts
    # Be careful with how you normalize!
    for c in counts.keys():
        for t in counts[c].keys():
            prob[c][t] = counts[c][t]/(sum(counts[c].values()))

    return prob

In [6]:
# RUN TO BUILD NGRAM MODEL

n = 4
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  4


In [7]:
#model[('<s>', )]['i']

In [8]:
def get_prob(model, context, w):
    ## FILL CODE
    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    # Bonus: You can also code an interpolation model this way
    if model[tuple(context)][w] != 0:
        return model[tuple(context)][w]
    else:
        return 0.4 * get_prob(model, context[1:], w) 

def perplexity(model, data, n):
    ## FILL CODE
    # Same as bigram.py
    perp = count = 0
    
   
    for sentence in data:
        context = sentence[:n-1]
        for word in sentence[n-1:]:
            perp += np.log(get_prob(model, context, word))
            if context:
                context.pop(0)
                context.append(word)
            count += len(sentence)
        return - perp/count

In [9]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n))

The perplexity is 0.5030118988940416


In [10]:
def get_proba_distrib(model, context):
    ## FILL CODE
    # code a recursive function over context
    # to find the longest available ngram 
    
    if sum(model[context].values()) != 0:
        return context
    else:
        return get_proba_distrib(model, context[1:])
    

def generate(model):
    ## FILL CODE
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    #   np.random.choice(x, 1, p = y)
    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    
    sentence = ["<s>"]
    
    while True :
        a = list(model[(sentence[-1], )].keys())
        prob = list(model[(sentence[-1], )].values())
        current_pred = np.random.choice(a, 1, \
                                     p = prob)
        sentence.append(current_pred[0])
        
        if current_pred[0] == '</s>':
            break
    
    return sentence

In [11]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'the', 'stairs.', '</s>']
